# Building a database for crime reports

In this project, we will build a database for storing data related to crimes that occurred in Boston. 

Our goal is to create a database named crimes_db with a table – boston_crimes – inside a schema named crimes with appropriate datatypes for storing the data from the boston.csv file. Also, we will create readonly and readwrite groups with the appropriate privileges. Lastly, we will create one user for each of these groups.

In [1]:
# Uncomment and run to run all jupyter notebook again without errors
# Drop database 

# import psycopg2
# conn = psycopg2.connect('dbname = dq user = dq')
# conn.autocommit = True
# cur = conn.cursor()
# cur.execute('DROP DATABASE crime_db')

# cur.execute('DROP GROUP readonly')
# cur.execute('DROP GROUP readwrite')
# cur.execute('DROP USER data_analyst')
# cur.execute('DROP USER data_cientist')
# conn.close()

## Creating the Crime Database

In [2]:
# Create database
import psycopg2
conn = psycopg2.connect('dbname = dq user = dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE DATABASE crime_db')
conn.close()

# Connect to the crime_db and create crimes schema
conn = psycopg2.connect('dbname = crime_db user = dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes')

## Obtaining the Column Names and Sample

We will gather some data about our crime dataset so that we can easily select the right datatypes to use in our table.

In [3]:
# Read CSV file
import csv
with open('boston.csv') as file:
    read = list(csv.reader(file))
    col_headers = read[0]
    first_row = read[1]
    
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

To identify the proper datatypes for the column, we'll create a function — get_col_set() — that, given the name of a CSV file and a column index (starting at 0), computes a Python set with all distinct values contained in that column.

Reasons why this function will be useful:
* Checking whether an enumerated datatype might be a good choice for representing a column.
* Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.

In [4]:
# Create function
def get_col_set(csv_filename, col_index):
    column = set()
    with open(csv_filename) as file:
        next(file)
        read = list(csv.reader(file))
        for row in read:
            column.add(row[col_index])
     
    return column

### Finding the Number of Different Values in each Column

In [5]:
# Find number of different values each column contains
len_diff_values = {}
for col_index in range(len(col_headers)):
    len_col = len(get_col_set('boston.csv', col_index))
    len_diff_values[col_headers[col_index]] = len_col
    
len_diff_values

{'date': 1177,
 'day_of_the_week': 7,
 'description': 239,
 'incident_number': 298329,
 'lat': 18177,
 'long': 18177,
 'offense_code': 219}

day_of_the_week column is a good candidate for enumerated datatype.

### Finding the Maximum Length

In [6]:
# Find longest element in each text column
col_values = {}
for col in range(len(col_headers)):
    column = list(get_col_set('boston.csv', col))
    if isinstance(column[0],str):
        max_value = max(column, key=len)
        col_values[col_headers[col]] = [max_value, len(max_value)]

max_len = 0
for key in col_values:
    print(key, col_values[key])
    max_len = max(max_len, col_values[key][1])
    
print(max_len)

description ['RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)', 58]
lat ['42.27173781', 11]
long ['-71.08670876', 12]
offense_code ['1830', 4]
incident_number ['227202', 6]
day_of_the_week ['Wednesday', 9]
date ['2018-05-30', 10]
58


## Creating the Table

In [7]:
# Read col_values
col_values

{'date': ['2018-05-30', 10],
 'day_of_the_week': ['Wednesday', 9],
 'description': ['RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)',
  58],
 'incident_number': ['227202', 6],
 'lat': ['42.27173781', 11],
 'long': ['-71.08670876', 12],
 'offense_code': ['1830', 4]}

In [8]:
# Read col_headers
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

To create our table, we defined the following datatypes:
* **DATE** for the data column.
* **ENUM** for the day_of_the_week column since it has only seven values.
* **VARCHAR** for the description column.
* **INTEGER** for the incident_number and the offense_code columns.
* **DECIMAL** for the lat and long columns.

In [9]:
# Get the day_of_the_week set 
weekdays = list(get_col_set('boston.csv', 4))
weekdays

['Friday', 'Wednesday', 'Thursday', 'Monday', 'Tuesday', 'Sunday', 'Saturday']

In [10]:
# Create enumerated datatype for the day_of_the_week column
enum_dtype = '''CREATE TYPE weekday AS ENUM (%s, %s, %s, %s, %s, %s, %s)'''
cur.execute(enum_dtype, weekdays)

In the Description column, the maximum length of text is 58. To be safe, we will limit the size of VARCHAR to 100.

In [11]:
# Create table inside schema
cur.execute('''CREATE TABLE crimes.boston_crimes (
                   incident_number INTEGER PRIMARY KEY,
                   offense_code INTEGER,
                   description VARCHAR(100),
                   date DATE,
                   day_of_the_week weekday,
                   lat DECIMAL,
                   long DECIMAL
            )''')

## Loading the Data

In [12]:
#Load data
with open('boston.csv') as file:
    cur.copy_expert('COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;', file)
    
# Test
cur.execute('SELECT * FROM crimes.boston_crimes LIMIT 5')
five_rows = cur.fetchall()
five_rows

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101'))]

## Revoking Public Privileges

Our goal is to create two user groups: readonly and readwrite. That's why we need to revoke privileges inherited from the public group and on the public schema first.

In [13]:
# Revoke privileges inherited
cur.execute('REVOKE ALL ON SCHEMA public FROM public;')
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public')

## Creating User Groups

Readonly group can only make SELECT queries while readwrite group can use SELECT, INSERT, DELETE, and UPDATE in queries. 

Users in those groups can:
* Connect to the database.
* Use the crimes schema.

We will create the groups with NOLOGIN because it is a group. Only user needs password to connec to the database. 

### Creating readonly group

In [14]:
# Create readonly group
cur.execute('CREATE GROUP readonly NOLOGIN')
# Grant privileges to readonly group
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly')
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;')

### Creating readwrite group

In [15]:
# Create readwrite group
cur.execute('CREATE GROUP readwrite NOLOGIN')

# Grant privileges to each group
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite')
cur.execute('''GRANT SELECT, INSERT, DELETE, UPDATE 
                  ON ALL TABLES IN SCHEMA crimes TO readwrite''')

## Creating Users

Lastly, we will create one user in each group.

### User data_analyst

In [16]:
# Creat data_analyst user and grant group readonly
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute('GRANT readonly to data_analyst;')

### User data_cientist

In [17]:
# Create data_cientist user and gran group readwrite
cur.execute("CREATE USER data_cientist WITH PASSWORD 'secret2'")
cur.execute("GRANT readwrite TO data_cientist")

## Test

Lastly, we will check the privileges.

In [18]:
# Check privileges in the table
cur.execute('''SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
                 FROM pg_roles 
                WHERE rolname in ('readonly', 'readwrite', 'data_analyst', 'data_cientist')
            ''')
roles = cur.fetchall()

print(['Role name', 'Superuser privileges', 'Can create roles', 'Can create databases', 'Can login'])
for role in roles:
    print(role)
    
cur.execute('''SELECT grantee, privilege_type
                 FROM information_schema.table_privileges
                WHERE table_name = 'boston_crimes'
                  AND grantee IN ('readonly', 'readwrite');
            ''')
privileges = cur.fetchall()

for row in privileges:
    print(row)
    
conn.close()

['Role name', 'Superuser privileges', 'Can create roles', 'Can create databases', 'Can login']
('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_cientist', False, False, False, True)
('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
